# Imports

In [1]:
import os
import pandas as pd
import numpy as np

from micom.media import minimal_medium

from micom import Community

basePath = "/home/arog/models"

# Dataset for Micom

In [2]:
# Load both datasets
# Replace these paths with your actual file paths
df_ids = pd.read_csv("/home/arog/Documents/GitHub/HGMB_Project/datasets/treated/models_final.csv")         # Dataset containing "Unique ID" column
df_data = pd.read_csv("/home/arog/Documents/GitHub/HGMB_Project/datasets/GFKB/GutFeelingKnowledgeBase-v4-Epilepsy_Data.csv")  # Dataset to filter based on "Unique ID"

# Set the first row as the new header
df_data.columns = df_data.iloc[0]
df_data = df_data[1:].reset_index(drop=True)

# Ensure the key column is correctly named
id_column = "Unique ID"
id2_column = "uniqueID"

# Merge instead of using isin
merged_df = df_ids.merge(df_data, left_on=id_column, right_on=id2_column, how='inner')
merged_df = merged_df.rename(columns={'UP name_x': 'UP name'})
merged_df = merged_df.drop(columns=['UP name_y']).iloc[:, :20]

# Convert to numeric (invalid strings become NaN)
merged_df["Mean_Control_Before_Diet"] = pd.to_numeric(merged_df["Mean_Control_Before_Diet"], errors="coerce")

# Then compute the relative abundance
merged_df["Relative_Abundance_Control_Before_Diet"] = (
    merged_df["Mean_Control_Before_Diet"] / merged_df["Mean_Control_Before_Diet"].sum()
)


merged_df.columns

Index(['Unique ID', 'UP name', 'NCBI taxonomy name',
       'Assembly ID (UP matched)', 'Present in GFKB v3 (Y/N)',
       'Present in GFKB v5 (Y/N)', 'BiGG Model Link', 'BiGG ID', 'uniqueID',
       'Lineage( full )', 'Median_Control_Before_Diet',
       'Mean_Control_Before_Diet', 'Standard_Deviation_Control_Before_Diet',
       'Interquartile_Range_Control_Before_Diet', 'Range_Control_Before_Diet',
       '0%_Control_Before_Diet', '25%_Control_Before_Diet',
       '50%_Control_Before_Diet', '75%_Control_Before_Diet',
       '100%_Control_Before_Diet', 'Relative_Abundance_Control_Before_Diet'],
      dtype='object')

In [3]:
basepath = "/home/arog/models/xml"

taxonomy = {
    "id": list(merged_df["BiGG ID"]),
    "file": [f"{basepath}/{x}.xml" for x in merged_df["BiGG ID"]],
    "abundance": list(merged_df["Relative_Abundance_Control_Before_Diet"])
}

taxonomy_df = pd.DataFrame(taxonomy)
taxonomy_df.to_csv("/home/arog/Documents/GitHub/HGMB_Project/datasets/taxonomy.csv", index=False)

# Micom Simulation

In [4]:
taxonomy = pd.read_csv("/home/arog/Documents/GitHub/HGMB_Project/datasets/taxonomy.csv", delimiter=",")
taxonomy

,id,file,abundance
0,iEC1356_Bl21DE3,/home/arog/models/xml/iEC1356_Bl21DE3.xml,0.025104
1,iEcSMS35_1347,/home/arog/models/xml/iEcSMS35_1347.xml,0.543917
2,iUTI89_1310,/home/arog/models/xml/iUTI89_1310.xml,0.043361
3,iECUMN_1333,/home/arog/models/xml/iECUMN_1333.xml,0.171923
4,iECSF_1327,/home/arog/models/xml/iECSF_1327.xml,0.072649
5,ic_1306,/home/arog/models/xml/ic_1306.xml,0.023221
6,iETEC_1333,/home/arog/models/xml/iETEC_1333.xml,0.057054
7,iNRG857_1313,/home/arog/models/xml/iNRG857_1313.xml,0.047165
8,iUMNK88_1353,/home/arog/models/xml/iUMNK88_1353.xml,0.015606


In [5]:
com = Community(taxonomy) # necessário instalar package especifica

Output()

In [6]:
# === Inspect Community Objective ===
print("Community objective expression:")
print(com.objective.expression)

Community objective expression:
1.0*community_objective


In [ ]:
# === FBA & pFBA Baseline ===
print("\nRunning baseline FBA...")
fba = com.optimize()
print("FBA community growth rate:", fba.growth_rate)
print("FBA community members:", fba.members)

print("\nRunning baseline pFBA...")
pfba = com.optimize(fluxes=True, pfba=True)
print("pFBA community growth rate:", pfba.growth_rate)
print("pFBA community members:", fba.members)



Running baseline FBA...
FBA community growth rate: 1.0329794581567353

Running baseline pFBA...
pFBA community growth rate: 1.03297759710055


In [ ]:
# === Cooperative Trade-Off Optimization ===
print("\nRunning cooperative trade-off (sol1) with min_growth = 0.0075...")
sol1 = com.cooperative_tradeoff(min_growth=0.0075, fluxes=True)
print("sol1 community growth:", sol1.growth_rate)
print("sol1 community members:", sol1.members)


Running cooperative trade-off (sol1) with min_growth = 0.0075...
sol1 community growth: 1.0328548173864236


In [9]:
# === Optional: run a second trade-off with per-taxon constraints
# min_growths = [0.0075] * len(com.taxa)  # Adjust manually if needed
# sol2 = com.cooperative_tradeoff(min_growth=min_growths, fluxes=True)

In [ ]:
# === Minimal Medium to Support 80% of Growth ===
print("\nComputing minimal medium for 80% growth...")
minMed = minimal_medium(com, 0.8, minimize_components=True)
print(minMed)


Computing minimal medium for 80% growth...


[05/31/25 19:02:31] WARNING  the MIP version of minimal media is extremely slow for models that large   ]8;id=678934;file:///home/arog/miniconda3/envs/micom/lib/python3.10/site-packages/micom/media.py\media.py]8;;\:]8;id=774400;file:///home/arog/miniconda3/envs/micom/lib/python3.10/site-packages/micom/media.py#72\72]8;;\
                             :(                                                                                    

EX_h2o_m       13.657999
EX_trp__L_m     1.121952
dtype: float64


In [11]:
# === Get Member Growth Rates (excluding medium pseudo-tax)
rates = sol1.members.growth_rate.drop("medium")

In [12]:
# === Constrain to Medium Supporting 10% Growth with Member Constraints ===
print("\nComputing restricted minimal medium for 10% of sol1 growth...")
med_1 = minimal_medium(com, 0.1 * sol1.growth_rate, min_growth=rates, exports=True)
com.medium = med_1


Computing restricted minimal medium for 10% of sol1 growth...


In [13]:
# === Rerun pFBA under new constrained medium ===
print("\nRe-running pFBA under constrained minimal medium...")
pfba_medium = com.optimize(fluxes=True, pfba=True)


Re-running pFBA under constrained minimal medium...


In [ ]:
# === Export Fluxes ===
sol1.fluxes.to_csv("/home/arog/Documents/GitHub/HGMB_Project/comm_results/fluxes_MICOM.csv", sep=";")
print("\nFluxes exported to 'fluxes_MICOM.csv'")

OSError: Cannot save file into a non-existent directory: '/home/arog/Documents/GitHub/HGMB_Project/results'

In [ ]:
# === Save Community Objective ===
print("Saving Community")
com.to_pickle("/home/arog/models/xml/community/micom_com")

Saving Community


In [ ]:
# === Print Final Results ===
print("\nMember growth rates under trade-off:")
print(sol1.members)

print("\nRestricted minimal medium (med_1):")
print(med_1)

print("\nMinimal medium (80%, minimized):")
print(minMed)


Member growth rates under trade-off:
                 abundance  growth_rate  reactions  metabolites
compartments                                                   
iEC1356_Bl21DE3   0.025104     0.082894       2740         1918
iECSF_1327        0.072649     0.224130       2742         1951
iECUMN_1333       0.171923     0.523835       2740         1935
iETEC_1333        0.057054     0.176889       2756         1962
iEcSMS35_1347     0.543917     1.652657       2746         1947
iNRG857_1313      0.047165     0.147372       2735         1943
iUMNK88_1353      0.015606     0.054238       2777         1969
iUTI89_1310       0.043361     0.135329       2725         1940
ic_1306           0.023221     0.076067       2726         1936
medium                 NaN          NaN        385          385

Restricted minimal medium (med_1):
EX_ca2_m         0.005376
EX_cl_m          0.005376
EX_co2_m       -21.089918
EX_cobalt2_m     0.000026
EX_cu2_m         0.000732
EX_glc__D_m     10.000000
EX